<a href="https://colab.research.google.com/github/gustavoramos82/Classificando-Fake-News-/blob/main/Notebooks/Modelagem/Aplicando_RNN_a_fake_news.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Aplicando uma Rede Neural Recorrente (RNN)

In [ ]:
# Importando as blibiotecas
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

from sklearn.model_selection import train_test_split

In [ ]:
# Lendo o dataset
news = pd.read_csv('/content/drive/MyDrive/Datasets/Fake news deep/Ouro/News.csv')
news.dropna(inplace=True)
news.drop('Unnamed: 0',axis=1,inplace=True)
news['label'] = news['label'].astype(int)
news.head()

,title,label
0,us budget fight loom republican flip fiscal sc...,0
1,us militari accept transgend recruit monday pe...,0
2,senior us republican senat let mr mueller job,0
3,fbi russia probe help australian diplomat tipo...,0
4,trump want postal servic charg much amazon shi...,0


In [ ]:
# Separando em treino e teste
x_train, x_test, y_train, y_test = train_test_split(news.title, news.label, test_size=0.2,
                                                    random_state=42)

In [ ]:
# Fazendo o pré-processamento do texto
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(x_train)
train_sequences = tokenizer.texts_to_sequences(x_train)

In [ ]:
vocab_size = len(tokenizer.word_index) + 1  # Tamanho do vocabulário (número de palavras únicas)
embedding_dim = 100  # Dimensão do vetor de embedding
max_length = max(len(seq) for seq in train_sequences)

In [ ]:
train_sequences = tf.keras.preprocessing.sequence.pad_sequences(train_sequences, maxlen=max_length) #aplicando padding

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_length))
model.add(LSTM(128))  # Camada LSTM com 128 unidades
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy',
                                                                     tf.keras.metrics.Precision(),tf.keras.metrics.Recall()])

In [ ]:
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 30, 100)           1777800   
                                                                 
 lstm (LSTM)                 (None, 128)               117248    
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,895,177
Trainable params: 1,895,177
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(train_sequences, y_train, epochs=5, batch_size=16)

Epoch 1/5
2115/2115 [==============================] - 38s 14ms/step - loss: 0.1616 - accuracy: 0.9345 - precision: 0.9457 - recall: 0.9205
Epoch 2/5
2115/2115 [==============================] - 15s 7ms/step - loss: 0.0657 - accuracy: 0.9760 - precision: 0.9795 - recall: 0.9718
Epoch 3/5
2115/2115 [==============================] - 14s 7ms/step - loss: 0.0369 - accuracy: 0.9869 - precision: 0.9888 - recall: 0.9848
Epoch 4/5
2115/2115 [==============================] - 15s 7ms/step - loss: 0.0218 - accuracy: 0.9918 - precision: 0.9936 - recall: 0.9899
Epoch 5/5
2115/2115 [==============================] - 16s 8ms/step - loss: 0.0125 - accuracy: 0.9954 - precision: 0.9965 - recall: 0.9943


In [ ]:
test_sequences = tokenizer.texts_to_sequences(x_test)
test_sequences = tf.keras.preprocessing.sequence.pad_sequences(test_sequences, maxlen=max_length)

In [ ]:
model.evaluate(test_sequences, y_test)

265/265 [==============================] - 2s 4ms/step - loss: 0.2167 - accuracy: 0.9522 - precision: 0.9504 - recall: 0.9517


[0.21672318875789642,
 0.9522458910942078,
 0.9503632187843323,
 0.951745867729187]